# Animating

Now that we have played a bit with positioning objects in 3D space, the next step will be animating objects in 3D space. The code that we will use is available in the downloaded `pypovray` package, see the [`template.py`](https://bitbucket.org/mkempenaar/pypovray/raw/master/template.py) module.

This document will show how to simply move a single `Sphere` from left to right (and vice-versa) in our scene. The document is concluded with a single assignment where we will simulate **TODO**. First, we will discuss what we need in order to be able to create a simple animation.

## Basics

While the idea of simply moving an object from left to right etc. sounds very straightforward, it involves quite a number of extra steps compared to just drawing a `Sphere` as we've done in the [`Povray Basics`](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/povray_basic.ipynb) document. Before we will start with examining the mentioned template file, let's discuss these extra steps and some of the specifics. 

Our previous examples resulting in a single *still* image. A movie or in our case an *animation* is a collection of images saved into a single file. Each separate image is called a **frame** in the animation. Our job is thus to create a series of images and feed it to some software that will make a movie out of it.

For our purpose, to create a frame, we construct a single scene and render this into an image file. For instance we have a `Sphere` located at `[`<font color='red'>0</font>`,` <font color='blue'>0</font>`,` <font color='green'>0</font>`]` and we want to move it to `[`<font color='red'>20</font>`,` <font color='blue'>10</font>`,` <font color='green'>0</font>`]`. Our first scene shows the `Sphere` at the starting coordinate and each following scene increases the `x`- and `y`-coordinate until it reached its destination position. Once we've defined *what* we want to show, we need to decide on the following very important **two** settings:

* How long will our animation run for?
* How *smooth* do we want our animation to be?
    
Let's say we want the `Sphere` to take exactly **2 seconds** to move from `[`<font color='red'>0</font>`,` <font color='blue'>0</font>`,` <font color='green'>0</font>`]` to `[`<font color='red'>20</font>`,` <font color='blue'>10</font>`,` <font color='green'>0</font>`]`, the *duration* will thus be `2`. The *smoothness* is defined by the number of *Frames Per Second*, generally denoted as **FPS**. It is often sufficient to render around **20 FPS** to get a smooth result, but this also depends on the movement of objects. If we look at the included `pypovray` configuration files we see that the `prototype.ini` defines 15 FPS and the `default.ini` 30 FPS. The reason why for prototyping (i.e. *testing*) the FPS is much lower is because this number defines how many frames we have to render and rendering is a *computationally expensive* operation. 

For this example we will set the FPS value to `30`. With these two numbers and the start- and end-coordinate of our `Sphere` we can answer the following questions:

* How many frames do we need to render, or *how many scenes do we need to construct*?
    * This can simply be answerd by multiplying the `duration` with the `FPS`: `2 * 30 = 60` scenes in total
* With what value do we need to increase the <font color='red'>x</font> and <font color='blue'>y</font> coordinates each scene?
    * Let's take the <font color='red'>x</font>-coordinate for example. We want it to move from `0` to `20` (i.e. move it to the right). We can take the difference between the end and the start and divide this by the *total number of scenes*:
        * `(20 - 0) / 60 = 20 / 60 =~ 0.33`
    * Concluding; for each scene, we need to increase the <font color='red'>x</font>-coordinate with ~ `0.33` to move the `Sphere` in 2 seconds from `0` to `20`.
    * For the <font color='blue'>y</font>-coordinate, this value is:
        * `(10 - 0) / 60 = 10 / 60 =~ 0.16`

In Python we store these values in a set of variables and use them where needed. The following code shows how we could do these calculations. How to *use* the values is shown thereafter.


In [ ]:
duration = 2 # in seconds
fps = 30
nframes = duration * fps # number of frames to render

# Define the 'path' of our Sphere
start = [0, 0, 0]
end = [20, 10, 0]

# Calculate 'step size' for each coordinate
x_distance_frame = (end[0] - start[0]) / nframes
y_distance_frame = (end[1] - start[1]) / nframes

## Template

Now that we have a good understanding on how to do some of the basic calculations and the settings that matter we can look at some code that actually creates an animation. This section takes code from the `template.py` file and discusses every line of code that is needed to create the following simple animation (shown in a *looped GIF* animation file):

<img src="https://bitbucket.org/mkempenaar/pypovray/raw/master/movies/template_md.gif" width=350>

As with all of our modules, we start with importing the necessary libraries and other modules. Compared to a previous example the only two new imports are `sys` (to use any command-line arguments) and the `SETTINGS` from the `povray` library, refer to the [`Installing and Configuring`](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/install_and_configure.ipynb) document for further details.

In [ ]:
import sys
from povray import povray, SETTINGS
from vapory import Sphere, Scene

Following the `import` statements is the definition of the `scene` function. We have seen this definition before with the only addition now being the `povray.default_floor` added to the `objects=[]` list. As you can see in the result above we added a 'floor' which gives some more depth to our animation. Do note again that the `scene` function has one paramter, the `step` which we do use in this example as we will see later below.

In [ ]:
def scene(step):
    ''' Returns the scene at step number (1 step per frame) '''
    
    # Some more code discussed below ...
    
    # Return the Scene object for rendering
    return Scene(povray.default_camera,
                 objects=[sphere, povray.default_ground, povray.default_light])

The first four statements in the `scene` function performs the first calculations we did above, namely calculating how many frames are created in the full animation and the distance travelled during the animation. By default we are using a configuration file which the `povray` library offers through the `SETTINGS` object and these contain both the *FPS* and the *duration* values that we use:

In [ ]:
nframes = SETTINGS.RenderFPS * SETTINGS.Duration

# Starting- and end-point (left side)
x_start = -10
x_end = 10
distance = x_end - x_start

Let's see what these default values are:

In [1]:
from povray import SETTINGS
nframes = SETTINGS.RenderFPS * SETTINGS.Duration
print("FPS: ", SETTINGS.RenderFPS, ", Duration: ", SETTINGS.Duration,
      "s, Total number of frames: ", nframes, sep='')

FPS: 30.0, Duration: 6.0s, Total number of frames: 180.0


This tells us that the final animation consists of **180** rendered scenes combined into a movie that takes **6** seconds. The next line of code shown below calculates the distance on the <font color='red'>x</font>-axis to travel each frame. The number **20** is actually the *end* minus the *start* which we will get to next. This number is then divided by the number of frames (180) as we've done in our example above. Finally, it is multiplied by **2** because in a single animation we move from the left to the right and fully back to the start.

In [ ]:
# Calculate distance to move at each step
distance_per_frame = (distance / nframes) * 2

The next part is a bit more complex as it contains an `if-else` statement, it uses the `step` parameter and all other calculated data. This is actually everything needed to define where in our scene the `Sphere` will be placed. First we'll show the whole code block and then break it down line by line.

In [ ]:
# Calculate new x-coordinate
if step < (nframes / 2):
    # Move from left to right (starting at x = -10)
    x_coord = x_start + step * distance_per_frame
else:
    # Move from right to left (starting at x = 10)
    x_coord = x_end - (step - (nframes / 2)) * distance_per_frame

The `if` statement does a simple check comparing if the value of `step` is smaller then `nframes / 2` (180 / 2 = 90). To understand what is happening here we need to know exactly what this `step` variable is and contains.

For clarity, this is what we know from the previous example: "*The step argument indicates where in the simulation we are, this can either be a timepoint in seconds (i.e. 1.45) or an integer defining the frame in a simulation (i.e. 30).*" In this animation we define `step` as the actual frame number in the animation with a range of `0..180`. Each scene that is rendered has a unique frame number and we can use this number to calculate where objects must be positioned *and* where in the animation we are!

The `if` statement then checks in which *half* of the animation we are (`if step < 90: ...`). If `step` is smaller then `90` we are in the first half, else in the second. If we are in the first half we move the sphere to the **right**, otherwise we move to the **left**.

In [ ]:
# Calculate new x-coordinate
if step < (nframes / 2):
    ...

*If* we are in the first half of the animation we move the `Sphere` to the *right*, i.e. we *increase* the <font color='red'>x</font>-coordinate. Since each step we move with the previously calculated `distance_per_frame` the amount travelled is calculated with `step * distance_per_frame`. This gives us a number to **add** to the starting position to get our final <font color='red'>x</font>-coordinate.

In [ ]:
# Move from left to right (starting at x = -10)
x_coord = x_start + step * distance_per_frame

If we are in the second part of the animation we want to move the `Sphere` to the *left*, i.e. we *decrease* the <font color='red'>x</font>-coordinate. However we need to *subtract* **90** frames from our `step` number to get the number of steps that we are in the second part of the animation. The final amount to travel is then *subtracted* from the `x_end` value to move to the left.

In [ ]:
# Move from right to left (starting at x = 10)
x_coord = x_end - (step - (nframes / 2)) * distance_per_frame

Here is a simple example (with reduced numbers) which you can use to experiment to get the above statement clear. If the numbers that are printed are coordinates, can you see how it moves from the left (`x_start`) to the right (`x_end`) and back to the left (`x_start`) again?

In [2]:
# Basic numbers
nframes = 20
distance_per_frame = 2
x_start = -10
x_end = 10

# Simulate the 'step' argument
for step in range(nframes+1):
    # If we are in the first half
    if step <= (nframes / 2):
        print(x_start + step * distance_per_frame, end=', ')
    # Or if we are in the second half
    else:
        print(int(x_end - (step - (nframes / 2)) * distance_per_frame), end=', ')

-10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 8, 6, 4, 2, 0, -2, -4, -6, -8, -10, 

We will now enter familiar territory, namely defining our `Sphere` object. The only thing to note is that we use the *calculated* `x_coord` value to position it on the <font color='red'>x</font>-axis.

In [ ]:
# Create sphere at calculated x-coordinate using default model
sphere = Sphere([x_coord, 0, 0], 2, povray.default_sphere_model)

The last statement in this function returns our `Scene` object including the `camera` and the `sphere`, `default_ground` and `default_light` objects.

In [ ]:
# Return the Scene object for rendering
return Scene(povray.default_camera,
             objects=[sphere, povray.default_ground, povray.default_light])

The rest of this module should read as familiar code (i.e. the `main` function and the `if __name__ == '__main__'` statement) which leaves us with the final function call to do the actual rendering. Previously we've used the `povray.make_frame()` function to render a single frame, now however we use something else to create an actual movie file.

The `render_scene_to_mp4` function takes two parameters:
* The `scene` function object that we constructed and
* the `time` paramter set to `False` indicating that the value of the `step` parameter denotes the integer *frame* instead of a timepoint in seconds.

In [ ]:
# Render as an MP4 movie
povray.render_scene_to_mp4(scene, time=False)

The two alternatives shown below can be used to either create a *GIF* output file (a low(er) quality animation which repeats itself) or render a single frame given a `step` number using the `make_frame` function:

In [ ]:
# Render as a GIF movie
povray.render_scene_to_gif(scene, time=False)

# Render step number 100
povray.make_frame(100, scene, time=False)

# Assignment